# Introductory Econometrics: A modern approach
---

## Chapter 16: Simultaneous Equations Models

> Book: 

    - WOOLDRIDGE, J. M.. Introdução à econometria: uma abordagem moderna. 3a ed. São Paulo: Pioneira Thomson Learning, 2006.

    - WOOLDRIDGE, Jeffrey M. Introductory econometrics: A modern approach. Cengage learning, 2015.

> LINK to codes correction:

http://www.upfie.net/downloads16.html

# Packages

In [46]:
import wooldridge as woo
import numpy as np
import pandas as pd
import linearmodels as lm
import linearmodels.iv as iv
import linearmodels.system as iv3
import statsmodels.formula.api as smf

# The Nature of Simultaneous Equations
---

> Cada equação do sistema tem uma interpretação causal.

## Oferta e demanda

Temos duas equações estruturais no mercado de trabalho, por exemplo:

**Oferta**

$$h_s = \alpha_1w + \beta_1z_1 + u_1$$

- $h_s$: horas de trabalho (ofertadas)

- $w$: salários em determinado setor (agricultura)

- $z_1$: outro fator *observável* que afeta a oferta de trabalho (salário na manufatura)

- $u_1$: termo do erro com fatores *não observáveis* que afetam a oferta de trabalho;

- $\alpha_1$ / $\beta_1$: coeficiente angular de impacto marginal (ou elasticidade na forma log) do salário na oferta de trabalho.

**Demanda**


$$
h_d = \alpha_2w + \beta_2z_2 + u_2
$$

- $h_s$: horas de trabalho (demandadas)

- $w$: salários em determinado setor (agricultura)

- $z_1$: outro fator *observável* que afeta a demanda de trabalho (tamanho da terra)

- $u_2$: termo do erro com fatores *não observáveis* que afetam a demanda de trabalho;

- $\alpha_2$ / $\beta_2$: coeficiente angular de impacto marginal (ou elasticidade na forma log) do salário / tamanho da terra na oferta de trabalho.

### **Importante**

**Nomenclaturas**

1. Variáveis endógenas;

2. Variáveis exógenas;

3. Erros estruturais;

4. **Equações estruturais**: REPRESENTAM o sistema e geralmente são DERIVAVÉIS da própria TEORIA ECONÔMICA.

> SEM $z_1$ e $z_2$ é IMPOSSÍVEL estimar as equações acima e diferenciá-las em qual representa oferta e qual representa demanda.

    - Em tese, sem essas variáveis, temos a mesma equação apenas com os w's e os u's.

Se aplicarmos esse sistema à elementos específicos (indivíduos, países, etc), a forma estrutural ganha o subscrito $i$ e, no equilíbrio:

$$h_{is} = h_{id}$$

**As equações são determinadas simultaneamente pelas mesmas variáveis! Ou seja, não podemos desenvolver, e nem há como estimar, um sistema de equações simultâneas se as próprias variáveis endógenas estiverem nas equações opostas!!!!**

#### **VALE LEMBRAR**

> Mesmo que as variáveis-alvo sejam determinadas simultaneamente, nem sempre faz sentido usar a abordagem SEM. Na prática, esse ferramental só pode ser aplicado a variáveis-chave que **POSSUEM INTERPRETAÇÃO CETERIS PARIBUS** independentemente da outra equação!

# Simultaneity Bias in OLS
--- 

Geralmente, **variáveis determinadas simultaneamente possuem correlação com os termos de erro**:

- A natureza desse viés é similar a da correlação de uma variável exógena com o termo do erro: se torna fonte de **endogeneidade**.

Logo:

> Estimadores de OLS serão viesados e incosistentes.

**Forma reduzida para DERVIAR e PROVAR a existências de viés e incosistência**

$$h_s = \alpha_1h_d + \beta_1z_1 + u_1$$

$$h_d = \alpha_2h_s + \beta_2z_2 + u_2$$

Substituindo valores, poderíamos chegar a uma forma reduzida:

$$h_s = \pi_1z_1 + \pi_2z_2 + v_2$$

Matemáticamente, os parâmetros reduzidos $\pi$ não são funções lineares dos parâmetros estruturais. Já os erros reduzidos são funções lineares do erro estrutural. 

Nesse formato, conseguimos estimar a equação em dois estágios por OLS.

### Contudo...

Não sabemos se $h_s$ e $h_d$ são correlacionados com os eros $u_2$ e $u_1$, respectivamente (correlação cruzada entre as endógenas e os erros).

Apenas se um dos $\alpha$'s for 0, podemos afirmar que não há esse tipo de correlação, mas essa seria uma hipótese muito forte, que em diversos casos podemos descartar.

Assim, em boa parte das aplicações, a aplicação de 2SLS (MQ20) produz estimadores viesados e inconsistentes.

**[VER FORMALIZAÇÃO COMPLETA NO WOOLDRIDGE, EM CASO DE DÚVIDAS]**

> **Chegamos na forma reduzida substituindo os elemntos da primeira forma estrutural na segunda forma estrutural, e vice-versa, solucionando a álgebra posteriormente.**

# Identifying and Estimating a Structural Equation
---

**PODEMOS APLICAR 2SLS PARA ESTIMAR EQUAÇÕES SIMULTÂNEAS**



Para podermos estimar as formas estruturais do modelos, precisamos solucionar o problema da identificação:

> **A primeira forma estrutural precisa de uma variável exógena que não está contida na segunda forma estrutural, e vice-versa, para que essa variável possa ser usada como instrumento para estimar a segunda forma estrutural.**

    - Novamente, isso vale para a lógica contrária, saindo da segunda para a primeira forma estrutural.

## Condição de ranking e condição de identificação

**Condição de identificação** (fraca): para usarmos variáveis instrumentais, é preciso que pelo menos uma variável exógena e não correlacionada com o termo do erro, seja exluída da forma estrutural do modelo.

**Condição de rank (posto)** (forte): para aplicarmos variáveis instrumentais, é preciso que pelo menos uma variável exógena omitida na primeira forma estrutural apareça na segunda forma estrutural com o coeficiente populacional diferente de zero, ou seja, $\beta_j \neq 0$.

Caso as condições se verifiquem e a equação seja identificável, podemos aplicar 2SLS: as variáveis instrumentais consistirão nas variáveis exógenas que estão em uma das formas estruturais, mas que não estão contidas na outra.

## Example 16.5: Labor Supply of Married, Working Women

In [47]:
mroz = woo.data('mroz')

# Retirando valores não observados da amostra

mroz.dropna(subset=['lwage'], inplace=True)
mroz.head()

,inlf,hours,kidslt6,kidsge6,age,educ,wage,repwage,hushrs,husage,...,faminc,mtr,motheduc,fatheduc,unem,city,exper,nwifeinc,lwage,expersq
0,1,1610,1,0,32,12,3.3540,2.65,2708,34,...,16310.0,0.7215,12,7,5.0,0,14,10.910060,1.210154,196
1,1,1656,0,2,30,12,1.3889,2.65,2310,30,...,21800.0,0.6615,7,7,11.0,1,5,19.499981,0.328512,25
2,1,1980,1,3,35,12,4.5455,4.04,3072,40,...,21040.0,0.6915,12,7,5.0,0,15,12.039910,1.514138,225
3,1,456,0,3,34,12,1.0965,3.25,1920,53,...,7300.0,0.7815,7,7,5.0,0,6,6.799996,0.092123,36
4,1,1568,1,2,31,14,4.5918,3.60,2000,32,...,27300.0,0.6215,12,14,9.5,1,7,20.100058,1.524272,49


Regressões com IV de ambas as formas estruturais

In [48]:
reg_iv165 = iv.IV2SLS.from_formula('hours ~ 1 + educ + age + kidslt6 + nwifeinc + [lwage ~ exper + expersq]', data=mroz)
results_reg_iv165 = reg_iv165.fit(cov_type='unadjusted', debiased=True)

reg2_iv165 = iv.IV2SLS.from_formula('lwage ~ 1 + educ + exper + expersq + [hours ~ age + kidslt6 + nwifeinc]', data=mroz)
results_reg2_iv165 = reg2_iv165.fit(cov_type='unadjusted', debiased=True)

print(results_reg_iv165.summary, results_reg2_iv165.summary, sep='\n')

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  hours   R-squared:                     -2.0076
Estimator:                    IV-2SLS   Adj. R-squared:                -2.0433
No. Observations:                 428   F-statistic:                    3.4410
Date:                Mon, Jul 04 2022   P-value (F-stat)                0.0046
Time:                        15:46:51   Distribution:                 F(5,422)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      2225.7     574.56     3.8737     0.0001      1096.3      3355.0
age           -7.8061     9.3780    -0.8324     0.40

Montando a tabela para printar os resultados

In [49]:
'''table_reg1 = pd.DataFrame({
    'b':round(results_reg_iv165.params, 4),
    'se':round(results_reg_iv165.std_errors, 4),
    't':round(results_reg_iv165.tstats, 4),
    'p-value':round(results_reg_iv165.pvalues, 4)
})

table_reg2 = pd.DataFrame({
    'b':round(results_reg2_iv165.params, 4),
    'se':round(results_reg2_iv165.std_errors, 4),
    't':round(results_reg2_iv165.tstats, 4),
    'p-value':round(results_reg2_iv165.pvalues, 4)
})

print(table_reg1, table_reg2, sep='\n\n')'''

"table_reg1 = pd.DataFrame({\n    'b':round(results_reg_iv165.params, 4),\n    'se':round(results_reg_iv165.std_errors, 4),\n    't':round(results_reg_iv165.tstats, 4),\n    'p-value':round(results_reg_iv165.pvalues, 4)\n})\n\ntable_reg2 = pd.DataFrame({\n    'b':round(results_reg2_iv165.params, 4),\n    'se':round(results_reg2_iv165.std_errors, 4),\n    't':round(results_reg2_iv165.tstats, 4),\n    'p-value':round(results_reg2_iv165.pvalues, 4)\n})\n\nprint(table_reg1, table_reg2, sep='\n\n')"

In [50]:
cor_u1u2 = np.corrcoef(results_reg_iv165.resids, results_reg2_iv165.resids)[0,1] # pegando apenas a correlação entre o erro da 
                                                                                 # primeira forma estrutural com a segunda
print(cor_u1u2)

-0.9037694117058851


## Example 16.6: Inflation and Openness

In [51]:
woo.data()

  J.M. Wooldridge (2016) Introductory Econometrics: A Modern Approach,
  Cengage Learning, 6th edition.

  401k       401ksubs    admnrev       affairs     airfare
  alcohol    apple       approval      athlet1     athlet2
  attend     audit       barium        beauty      benefits
  beveridge  big9salary  bwght         bwght2      campus
  card       catholic    cement        census2000  ceosal1
  ceosal2    charity     consump       corn        countymurders
  cps78_85   cps91       crime1        crime2      crime3
  crime4     discrim     driving       earns       econmath
  elem94_95  engin       expendshares  ezanders    ezunem
  fair       fertil1     fertil2       fertil3     fish
  fringe     gpa1        gpa2          gpa3        happiness
  hprice1    hprice2     hprice3       hseinv      htv
  infmrt     injury      intdef        intqrt      inven
  jtrain     jtrain2     jtrain3       kielmc      lawsch85
  loanapp    lowbrth     mathpnl       meap00_01   meap01
  meap93    

In [52]:
openness = woo.data('openness')
openness.head()

,open,inf,pcinc,land,oil,good,lpcinc,lland,lopen,linf,opendec,linfdec
0,31.400000,9.9,1998,919595.0,0,0,7.599902,13.731688,3.446808,2.292535,0.314,-2.312635
1,9.400000,117.0,4342,1072067.0,0,1,8.376090,13.885099,2.240710,4.762174,0.094,0.157004
2,16.700001,9.5,8349,2966150.0,0,1,9.029897,14.902776,2.815409,2.251292,0.167,-2.353878
3,35.599998,5.0,8230,32375.0,0,1,9.015541,10.385141,3.572345,1.609438,0.356,-2.995732
4,91.000000,4.8,9185,240.0,1,0,9.125327,5.480639,4.510859,1.568616,0.910,-3.036554


In [53]:
reg_ols166 = smf.ols('open ~ np.log(pcinc) + lland', data=openness)
results_reg_ols166 = reg_ols166.fit()
print(results_reg_ols166.summary())

                            OLS Regression Results                            
Dep. Variable:                   open   R-squared:                       0.449
Model:                            OLS   Adj. R-squared:                  0.439
Method:                 Least Squares   F-statistic:                     45.17
Date:                Mon, 04 Jul 2022   Prob (F-statistic):           4.45e-15
Time:                        15:46:53   Log-Likelihood:                -488.44
No. Observations:                 114   AIC:                             982.9
Df Residuals:                     111   BIC:                             991.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept       117.0845     15.848      7.388

In [54]:
reg_iv166 = iv.IV2SLS.from_formula('inf ~ 1 + np.log(pcinc) + [open ~ lland]', data=openness)
results_reg_iv166 = reg_iv166.fit(cov_type='unadjusted', debiased=True)
print(results_reg_iv166.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                    inf   R-squared:                      0.0309
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0134
No. Observations:                 114   F-statistic:                    2.7900
Date:                Mon, Jul 04 2022   P-value (F-stat)                0.0657
Time:                        15:46:53   Distribution:                 F(2,111)
Cov. Estimator:            unadjusted                                         
                                                                              
                               Parameter Estimates                               
               Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------
Intercept         26.899     15.401     1.7466     0.0835     -3.6192      57.418
np.log(pcinc)     0.3758     2.0151     

# Systems with More Than Two Equations
---

Em sistemas simultaneos com mais de duas equações, se conseguirmos identificar as equações, podemos estimar também por meio dos Mínimos Quadrados em dois Estágios (2SLS).

> ***Condição de ordem (o número de variáveis exlcuídas/omitidas deve ser pelo menos o mesmo do número de variáveis endógenas) é necessária, mas não suficiente!!!***

Há a necessidades dos coeficientes das variáveis exluídas de uma equação, e que aparecem em outras partes do sistema, tenha coeficiente populacional diferente de zero: $\beta_{ij} \neq 0$

Ou seja, não depende apenas da existência de variáveis excluídas em outras equações do sistema, mas também dos valores dos parâmetros dessas variáveis.

> Para aplicar a condição de posto (rank), precisamos de uma álgebra um pouco mais exaustiva, o que significa que em muitas aplicações, pesquisadores assumem por intuição que a satisfação da condição de ordem já é suficiente para considerarmos as equações como identificadas.

A nomenclatura de equações **sobreidentificadas**, **exatamente identificada** e **subidentificada/não identificada** aplica-se aos sistemas de equações simultâneas:

1. Número de instrumentos maior do que o número de variáveis exógenas excluídas;

2. Número de instrumentos igual ao número de variáveis exógenas excluídas;

3. Número de instrumentos menor do que o número de variáveis exógenas excluídas.

**Apenas no último caso não vamos conseguir identificar e nem estimar a equação.**

# Simultaneous Equations Models with Time Series

> Um das principais e primeiras aplicações dos sistemas de equações simultâneas foi usada para descrever a economia como um todo, por meio da interpretação básica do modelo Keynesiano.

Quando podemos coletar dados referentes à séries temporais, podemos estimar as equações simultâneas adicionando lags de variáveis endógenas como instrumentos. Contudo, para que seja possível realizar os trabalhos de inferência, temos que contar que as variáveis tenham **dependência fraca em relação aos lags**, o que geralmente pode não acontecer. 

> Exemplo do Modelo Keynesiano citado acima!

Ainda, vale lembrar que com séries temporais, podemos contar com sazonalidade, persistência e tendências.

> Mas esses problemas não tornam os SEM inapropriados para lidar com séries temporais.

Uma das soluções para estimar SEMs com séries temporais que possuem essas características é a definição das equações com primeiras, segundas ou demais diferenças.

Ainda, o uso de **dados desagregados** pode auxiliar na estimação dessas equações em que achar instrumentos se torna uma tarefa complicada devido a endogeneidade. 

## Example 16.7: Testing the Permanent Income Hypothesis

In [55]:
comsup = woo.data('consump')
comsup.head()

,year,i3,inf,rdisp,rnondc,rserv,pop,y,rcons,c,...,gy,gc_1,gy_1,r3_1,lc_ly,lc_ly_1,gc_2,gy_2,r3_2,lc_ly_2
0,1959,3.41,0.7,1530.099976,606.299988,687.400024,177830.0,8604.285156,1293.699951,7274.925293,...,NaN,NaN,NaN,NaN,-0.167827,NaN,NaN,NaN,NaN,NaN
1,1960,2.93,1.7,1565.400024,615.400024,717.400024,180671.0,8664.368164,1332.800049,7376.945312,...,0.006959,NaN,NaN,2.71,-0.160860,-0.167827,NaN,NaN,NaN,NaN
2,1961,2.38,1.0,1615.800049,626.700012,746.500000,183691.0,8796.293945,1373.199951,7475.597168,...,0.015111,0.013926,0.006959,1.23,-0.162686,-0.160860,NaN,NaN,2.71,-0.167827
3,1962,2.78,1.0,1693.699951,646.500000,783.400024,186538.0,9079.651367,1429.900024,7665.462402,...,0.031706,0.013285,0.015111,1.38,-0.169312,-0.162686,0.013926,0.006959,1.23,-0.160860
4,1963,3.16,1.3,1755.500000,660.000000,818.700012,189242.0,9276.482422,1478.699951,7813.804199,...,0.021446,0.025081,0.031706,1.78,-0.171591,-0.169312,0.013285,0.015111,1.38,-0.162686


In [56]:
comsup.dropna(subset=['gc', 'gy', 'r3', 'gy_1'], inplace=True)

In [57]:
pih_reg = iv.IV2SLS.from_formula('gc ~ 1 + [gy + r3 ~ gc_1 + gy_1 + r3_1]', data=comsup)
results_pih_reg = pih_reg.fit(cov_type='unadjusted', debiased=True)
print(results_pih_reg)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                     gc   R-squared:                      0.6779
Estimator:                    IV-2SLS   Adj. R-squared:                 0.6578
No. Observations:                  35   F-statistic:                    9.5862
Date:                Mon, Jul 04 2022   P-value (F-stat)                0.0005
Time:                        15:46:54   Distribution:                  F(2,32)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      0.0081     0.0032     2.4931     0.0180      0.0015      0.0146
gy             0.5862     0.1346     4.3559     0.00

> Se as formas estruturais contiverem tendências temporais, os lags das variáveis podem ser usados como seus próprios instrumentos, sempre levando em conta a correlação serial do termo do erro como um impeditivo para inferência, caso ocorra. 

# Simultaneous Equations Models with Panel Data
---

SEM também podem ser aplicados a dados em painel, para incluirmos os efeitos fixos e aleatórios sobre os indivíduos de uma amostra com os métodos que vimos anteriormente. 

Nesse caso, também aplicamos variáveis instrumentais para controlar para variáveis exógenas que podem provocar endogeneidade. Assim:

- O desafio é encontrar instrumentos que não sejam efeitos fixos: **eles precisam variar no tempo**.

Em uma análise geral, podemos:

1. Permitir que os efeitos não observáveis estejam correlacionados com todas as variáveis explicativas;

Contudo:

2. DEVEMOS LEMBRAR QUE os erros estruturais idiossincráticos NÃO PODEM ser correlacionados com as variáveis exógenas (as que não causam endogeneidade).

> VALE LEMBRAR: nesses casos, não podemos estimar via OLS, dado que o erro composto ($\alpha_{i1} + u_{i1}$) estará parcialmente correlacionado com as variáveis explicativas. 

Usaremos então a primeira diferenciação ou uma redução de tempo, para, por definição, eliminarmos a possível correlação:

- O único impasse será encontrar os instrumentos para as variáveis que são fonte de endogeneidade.

Testes de autocorrelação dos erros podem ser realizados a partir da obtenção dos resíduos e uma regressão em suas lags:

- 2SLS e suas estatísticas t serão válidas para testar essa hipótese.

Além da diferenciação, a estimação de SEM com dados em painel também pode ser realizada pela ***transformação de efeitos fixos*** (além da diferenciação), aliada à 2SLS.  

Para identificações no tempo com dummies, podemos usá-las como seus próprios instrumentos.

# ***Computer Exercises***
---

## C1

In [58]:
smoke = woo.data('smoke')
smoke.head()

,educ,cigpric,white,age,income,cigs,restaurn,lincome,agesq,lcigpric
0,16.0,60.506001,1,46,20000,0,0,9.903487,2116,4.102743
1,16.0,57.882999,1,40,30000,0,0,10.308952,1600,4.058424
2,12.0,57.664001,1,58,30000,3,0,10.308952,3364,4.054633
3,13.5,57.882999,1,30,20000,0,0,9.903487,900,4.058424
4,10.0,58.320000,1,17,20000,0,0,9.903487,289,4.065945


### i)

Assumindo que a equação apresentada seja a forma estrutural, podemos assumir que 100*$\beta_1$ é o efeito marginal, em porcentagem, do consumo de cigarros sobre a renda.

### ii) 

Para $\gamma_5$ espera-se um sinal negativo ($\gamma_5 \leq 0$), dado que um maior preço do cigarro poderia reduzir a demanda por cigarros. Contudo, se a amostra for composta com diversos fumantes viciados, pode ser que o coeficiente seja próximo de zero.

Para $\gamma_6$, espera-se um sinal negativo ($\gamma_6 \leq 0$), indicando que a presenla de restrições sobre o fumo em restaurantes impacte negativamente o consumo.

### iii) 

A equação estrutural apresentada será identificada se:

1. A forma reduzida precisa conter uma variável exógena omitida da forma estrutural e que possa ser usada como instrumento --> Condição de identificação;

2. A variável omitida destacada no item 1 precisa ter o coeficiente populacional estatisticamente diferente de zero na forma reduzida --> Condição de posto;

Na prática, precisamos que $\gamma_5$ e $\gamma_6$ sejam **diferentes de zero**. 

### iv)

Estimando a forma estrutural com OLS:

$$
log(income) = \beta_0 + \beta_1cigs + \beta_2educ + \beta_3age + \beta_4age^2 + u_1
$$

In [59]:
smoke_ols = smf.ols('lincome ~ cigs + educ + age + agesq', data=smoke)
results_smoke_ols = smoke_ols.fit()
print(results_smoke_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                lincome   R-squared:                       0.165
Model:                            OLS   Adj. R-squared:                  0.161
Method:                 Least Squares   F-statistic:                     39.61
Date:                Mon, 04 Jul 2022   Prob (F-statistic):           2.68e-30
Time:                        15:46:55   Log-Likelihood:                -798.50
No. Observations:                 807   AIC:                             1607.
Df Residuals:                     802   BIC:                             1630.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.7954      0.170     45.741      0.0

Sobre $\beta_1$, podemos dizer que o coeficiente não é estatisticamente significante nem ao nível de significância de 10%, indicando que o número de cigarros fumados por dia não influencia na renda de um indivíduo em idade ativa. Mesmo assim, ao analisar apenas o valor do coeficiente, poderíamos dizer que o consumo de cigarros tem efeito positivo sobre a renda, sendo que o consumo de 1 cigarro a mais aumentaria em (100*0,0017 = 0,17%)  0.17% a renda deste indivíduo.

Vale lembrar que estimar via OLS nos faz ignorar a possível simultaneidade existente entre renda e consumo de cigarros.

### v)

Estimando a forma reduzida via OLS: 

$$
cigs = \gamma_0 + \gamma_1log(income) + \gamma_2educ + \gamma_3age + \gamma_4age^2 + \gamma_5log(cigpric) + \gamma_6restaurn + u_2
$$

In [60]:
smoke.head()

,educ,cigpric,white,age,income,cigs,restaurn,lincome,agesq,lcigpric
0,16.0,60.506001,1,46,20000,0,0,9.903487,2116,4.102743
1,16.0,57.882999,1,40,30000,0,0,10.308952,1600,4.058424
2,12.0,57.664001,1,58,30000,3,0,10.308952,3364,4.054633
3,13.5,57.882999,1,30,20000,0,0,9.903487,900,4.058424
4,10.0,58.320000,1,17,20000,0,0,9.903487,289,4.065945


In [61]:
cigs_ols = smf.ols('cigs ~ educ + age + agesq + lcigpric + restaurn', data=smoke)
results_cigs_ols = cigs_ols.fit()
print(results_cigs_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                   cigs   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.045
Method:                 Least Squares   F-statistic:                     8.610
Date:                Mon, 04 Jul 2022   Prob (F-statistic):           5.86e-08
Time:                        15:46:56   Log-Likelihood:                -3237.0
No. Observations:                 807   AIC:                             6486.
Df Residuals:                     801   BIC:                             6514.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.5801     23.696      0.067      0.9

Enquanto o preço do cigarro [log(cigpric)] não é estatisticamente significante, a proibição do fumo em restaurantes é significante. Nesse caso, pessoas que vivem em estados com proibição de fumo nesses ambientes tendem a fumar quase que 3 cigarros a menos por dia.

In [62]:
cigs_ols_rest_price = smf.ols('cigs ~ lcigpric + restaurn', data=smoke)
results_cigs_ols_restpric = cigs_ols_rest_price.fit()
print(results_cigs_ols_restpric.summary())

                            OLS Regression Results                            
Dep. Variable:                   cigs   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     3.077
Date:                Mon, 04 Jul 2022   Prob (F-statistic):             0.0467
Time:                        15:46:57   Log-Likelihood:                -3255.0
No. Observations:                 807   AIC:                             6516.
Df Residuals:                     804   BIC:                             6530.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.5384     24.045      0.438      0.6

Poderíamos tirar o preço do cigarro da análise, mas essa variável é conjuntamente significante junto das restrições de fumo em restaurantes, pelo teste F de Snedecor, vamos deixá-la no modelo.

### vi)

Estimando a forma estrutural por meio de 2SLS usando o preço do cigarro e as restrições em restaurantes como instrumentos para a quantidade de cigarros fumados em um dia.

In [63]:
smoke_iv = iv.IV2SLS.from_formula('lincome ~ 1 + [cigs ~ lcigpric + restaurn] + educ + age + agesq', data=smoke)
results_smoke_iv = smoke_iv.fit(cov_type='unadjusted', debiased=True)
print(results_smoke_iv.summary, results_smoke_ols.summary(), sep='\n\n')

                          IV-2SLS Estimation Summary                          
Dep. Variable:                lincome   R-squared:                     -0.5169
Estimator:                    IV-2SLS   Adj. R-squared:                -0.5245
No. Observations:                 807   F-statistic:                    22.311
Date:                Mon, Jul 04 2022   P-value (F-stat)                0.0000
Time:                        15:46:57   Distribution:                 F(4,802)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      7.7809     0.2299     33.850     0.0000      7.3297      8.2321
age            0.0938     0.0239     3.9331     0.00

Podemos observar que o valor de $\beta_1$ é bem distinto entre os métodos. A começar pelo sinal: enquanto por OLS o estimador nos dá um valor positivo, mas estatisticamente insignificante, em 2SLS e IV's temos que o seu valor é negativo e indica que a renda do indivíduo tende a cair (100 * 0.0421 = 4.21%) 4.21% a cada cigarro fumado a mais por dia, ao contrário do aumento em 0.17% no exercício anterior por OLS. 

Vale lembrar que mesmo estimando por 2SLS, o estimador quase é significante ao nível de 10%. Vale lembrar, contudo, que o intervalo de confiança para testar a hipótese é mais amplo que em OLS, o que pode influenciar na significância. Mas como, por propriedade o estimador é consistente, podemos levantar novamenta a hipótese de sua relevância em um número significativo de indivíduos. 

### vii)

Assumir que as restrições de fumo e os preços do cigarro são exógenas na forma estrutural pode ser problemático. Isso porque podem haver efeitos fixos por estado, por exemplo: a renda média do estado pode influenciar se há ou não restrições em restaurantes (após controlar para outros fatores) ou até mesmo legislações que sobretaxem os cigarros para controlar suas externalidades negativas. Pode ser que em um estado de menor renda seja menos provável que essas legislações sejam implementadas. Nesse caso, precisaríamos controlar para os efeitos fixos de estados.  

## C2

In [64]:
mroz = woo.data('mroz')
mroz.head()

,inlf,hours,kidslt6,kidsge6,age,educ,wage,repwage,hushrs,husage,...,faminc,mtr,motheduc,fatheduc,unem,city,exper,nwifeinc,lwage,expersq
0,1,1610,1,0,32,12,3.3540,2.65,2708,34,...,16310.0,0.7215,12,7,5.0,0,14,10.910060,1.210154,196
1,1,1656,0,2,30,12,1.3889,2.65,2310,30,...,21800.0,0.6615,7,7,11.0,1,5,19.499981,0.328512,25
2,1,1980,1,3,35,12,4.5455,4.04,3072,40,...,21040.0,0.6915,12,7,5.0,0,15,12.039910,1.514138,225
3,1,456,0,3,34,12,1.0965,3.25,1920,53,...,7300.0,0.7815,7,7,5.0,0,6,6.799996,0.092123,36
4,1,1568,1,2,31,14,4.5918,3.60,2000,32,...,27300.0,0.6215,12,14,9.5,1,7,20.100058,1.524272,49


### i)

Forma estrutural:

$$
hours = \alpha_1log(wage) + \beta_{10} + \beta_{11}educ + \beta_{12}age + \beta_{13}kidslt6 + \beta_{14}nwifeinc + u_1
$$

Forma reduzida:

$$
log(wage) = \alpha_2hours + \beta_{20} + \beta_{21}educ + \beta_{22}exper + \beta_{23}exper^2 + u_2
$$

REESTIMANDO A FORMA ESTRUTURAL EM:

$$
log(hours) = \alpha_1log(wage) + \beta_{10} + \beta_{11}educ + \beta_{12}age + \beta_{13}kidslt6 + \beta_{14}nwifeinc + u_1
$$

In [65]:
mroz.dropna(subset=['lwage'], inplace=True)

In [66]:
loghours_iv = iv.IV2SLS.from_formula('np.log(hours) ~ 1 + [lwage ~ exper + expersq] + educ + age + kidslt6 + nwifeinc', data=mroz)
results_loghours_iv = loghours_iv.fit(cov_type='unadjusted', debiased=True)
print(results_loghours_iv.summary, results_reg_iv165)

                          IV-2SLS Estimation Summary                          
Dep. Variable:          np.log(hours)   R-squared:                     -1.7762
Estimator:                    IV-2SLS   Adj. R-squared:                -1.8091
No. Observations:                 428   F-statistic:                    4.8105
Date:                Mon, Jul 04 2022   P-value (F-stat)                0.0003
Time:                        15:46:58   Distribution:                 F(5,422)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      8.3702     0.6890     12.148     0.0000      7.0159      9.7246
age           -0.0135     0.0112    -1.2026     0.22

Ao estimarmos usado log(hours) como variável dependente, a elasticidade preço (salário) daoferta de trabalho é de 1.99%. O número é ainda maior do que o 1.26% [1639.6 / 1303 (média de horas trabalhadas na amostra) = 1.26%] que estimamos com hours sendo a variável dependente. 

Um dos possíveis motivos para isso ocorrer, é que o logaritmo de horas trabalhadas só pode ser calculado para valores acima de zero, o que nos faz excluir esses dados da amostra para realizar a transformação log.

### ii) 

Permitindo que educ seja endógena e usando a educação da mãe e do pai como instrumentos

In [67]:
loghours_iv2 = iv.IV2SLS.from_formula('np.log(hours) ~ 1 + [lwage + educ ~ exper + expersq + motheduc + fatheduc] + age + kidslt6 + nwifeinc', data=mroz)
results_loghours_iv2 = loghours_iv2.fit(cov_type='unadjusted', debiased=True)
print(results_loghours_iv2.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:          np.log(hours)   R-squared:                     -1.4820
Estimator:                    IV-2SLS   Adj. R-squared:                -1.5114
No. Observations:                 428   F-statistic:                    5.1828
Date:                Mon, Jul 04 2022   P-value (F-stat)                0.0001
Time:                        15:46:58   Distribution:                 F(5,422)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      7.2608     1.0194     7.1226     0.0000      5.2570      9.2645
age           -0.0116     0.0106    -1.0959     0.27

In [68]:
u_iv2 = results_loghours_iv2.resids
mroz['resids'] = u_iv2
reg_resid_iv2 = smf.ols('resids ~ exper + expersq + motheduc + fatheduc + age + kidslt6 + nwifeinc', 
                                        data=mroz).fit()
print(reg_resid_iv2.summary())

                            OLS Regression Results                            
Dep. Variable:                 resids   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.016
Method:                 Least Squares   F-statistic:                   0.06264
Date:                Mon, 04 Jul 2022   Prob (F-statistic):               1.00
Time:                        15:46:59   Log-Likelihood:                -787.61
No. Observations:                 428   AIC:                             1591.
Df Residuals:                     420   BIC:                             1624.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0042      0.606      0.007      0.9

### iii)

Teste de sobreidentificação (Teste de Sargan)

$H_0$: não está sobreidentificado

$H_1$: Modelo sobreidentificado

In [69]:
# calculations for test:
import scipy.stats as stats

r2 = reg_resid_iv2.rsquared
n = reg_resid_iv2.nobs
teststat = n * r2
pval = 1 - stats.chi2.cdf(teststat, 2)
pval, teststat

(0.799964724573692, 0.4463752931385523)

In [70]:
print(results_loghours_iv2.sargan, results_loghours_iv.sargan, results_loghours_iv2.wu_hausman(), results_loghours_iv.wu_hausman(), sep='\n\n')

Sargan's test of overidentification
H0: The model is not overidentified.
Statistic: 0.4464
P-value: 0.8000
Distributed: chi2(2)

Sargan's test of overidentification
H0: The model is not overidentified.
Statistic: 0.0685
P-value: 0.7935
Distributed: chi2(1)

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 20.0137
P-value: 0.0000
Distributed: F(2,420)

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 41.2026
P-value: 0.0000
Distributed: F(1,421)


Pelos testes de Wu-Hausman e de Sargan, podemos concluir que nossos modelos não estão sobreidentificados e que todas as variáveis endógenas são de fato endógenas.

## C3 

PULAR DEVIDO AOS ERROS

### i)

In [71]:
openness = woo.data('openness')
openness.head()

,open,inf,pcinc,land,oil,good,lpcinc,lland,lopen,linf,opendec,linfdec
0,31.400000,9.9,1998,919595.0,0,0,7.599902,13.731688,3.446808,2.292535,0.314,-2.312635
1,9.400000,117.0,4342,1072067.0,0,1,8.376090,13.885099,2.240710,4.762174,0.094,0.157004
2,16.700001,9.5,8349,2966150.0,0,1,9.029897,14.902776,2.815409,2.251292,0.167,-2.353878
3,35.599998,5.0,8230,32375.0,0,1,9.015541,10.385141,3.572345,1.609438,0.356,-2.995732
4,91.000000,4.8,9185,240.0,1,0,9.125327,5.480639,4.510859,1.568616,0.910,-3.036554


## C4

In [77]:
consump = woo.data('consump')
print(consump.head())

   year    i3  inf        rdisp      rnondc       rserv       pop  \
0  1959  3.41  0.7  1530.099976  606.299988  687.400024  177830.0   
1  1960  2.93  1.7  1565.400024  615.400024  717.400024  180671.0   
2  1961  2.38  1.0  1615.800049  626.700012  746.500000  183691.0   
3  1962  2.78  1.0  1693.699951  646.500000  783.400024  186538.0   
4  1963  3.16  1.3  1755.500000  660.000000  818.700012  189242.0   

             y        rcons            c  ...        gy      gc_1      gy_1  \
0  8604.285156  1293.699951  7274.925293  ...       NaN       NaN       NaN   
1  8664.368164  1332.800049  7376.945312  ...  0.006959       NaN       NaN   
2  8796.293945  1373.199951  7475.597168  ...  0.015111  0.013926  0.006959   
3  9079.651367  1429.900024  7665.462402  ...  0.031706  0.013285  0.015111   
4  9276.482422  1478.699951  7813.804199  ...  0.021446  0.025081  0.031706   

   r3_1     lc_ly   lc_ly_1      gc_2      gy_2  r3_2   lc_ly_2  
0   NaN -0.167827       NaN       NaN       

### i)

In [78]:
consump.dropna(subset=['gc', 'gy', 'r3', 'gy_1'], inplace=True)

pih_reg = iv.IV2SLS.from_formula('gc ~ 1 + [gy + r3 ~ gc_1 + gy_1 + r3_1]', data=consump)
results_pih_reg = pih_reg.fit(cov_type='unadjusted', debiased=True)
print(results_pih_reg)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                     gc   R-squared:                      0.6779
Estimator:                    IV-2SLS   Adj. R-squared:                 0.6578
No. Observations:                  35   F-statistic:                    9.5862
Date:                Mon, Jul 04 2022   P-value (F-stat)                0.0005
Time:                        15:52:45   Distribution:                  F(2,32)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      0.0081     0.0032     2.4931     0.0180      0.0015      0.0146
gy             0.5862     0.1346     4.3559     0.00

In [80]:
print(results_pih_reg.sargan)

Sargan's test of overidentification
H0: The model is not overidentified.
Statistic: 2.1463
P-value: 0.1429
Distributed: chi2(1)


O modelo não está sobreidentificado.

### ii)